<a href="https://colab.research.google.com/github/muthuijack/FLIGHT-/blob/main/mood_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("msambare/fer2013")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'fer2013' dataset.
Path to dataset files: /kaggle/input/fer2013


In [3]:
# List files in dataset directory
print(os.listdir(path))


['test', 'train']


In [4]:
print("Dataset path:", path)
print("Files inside:", os.listdir(path))


Dataset path: /kaggle/input/fer2013
Files inside: ['test', 'train']


In [5]:
print("Subfolders:", os.listdir(os.path.join(path)))


Subfolders: ['test', 'train']


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = os.path.join(path, "train")
test_dir = os.path.join(path, "test")

# Data augmentation & normalization
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Create train and test generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    color_mode="grayscale",
    class_mode="categorical",
    batch_size=64
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    color_mode="grayscale",
    class_mode="categorical",
    batch_size=64
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [7]:
print("Class indices:", train_generator.class_indices)


Class indices: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


In [8]:
from PIL import Image

def check_images(folder):
    for root, _, files in os.walk(folder):
        for file in files:
            try:
                img = Image.open(os.path.join(root, file))
                img.verify()
            except:
                print("Corrupt image removed:", file)
                os.remove(os.path.join(root, file))


In [9]:
#from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths
train_dir = os.path.join(path, "train")
test_dir = os.path.join(path, "test")

# Data augmentation and normalization
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Train generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    color_mode="grayscale",
    class_mode="categorical",
    batch_size=64,
    shuffle=True
)

# Test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    color_mode="grayscale",
    class_mode="categorical",
    batch_size=64,
    shuffle=False
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Define the CNN model
model = Sequential()

# Convolutional layers
model.add(Conv2D(64, (3,3), activation='relu', input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Fully connected layers
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # 7 moods

# Compile
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 46, 46, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 21, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 21, 21, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     1,048,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,422,087 (5.42 MB)

 Trainable params: 1,421,191 (5.42 MB)

 Non-trainable params: 896 (3.50 KB)

In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Callbacks
checkpoint = ModelCheckpoint("best_mood_model.h5", monitor='val_accuracy',
                             save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator,
    callbacks=[checkpoint, early_stop]
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step - accuracy: 0.2327 - loss: 2.2504
Epoch 1: val_accuracy improved from -inf to 0.24882, saving model to best_mood_model.h5


449/449 ━━━━━━━━━━━━━━━━━━━━ 131s 270ms/step - accuracy: 0.2328 - loss: 2.2495 - val_accuracy: 0.2488 - val_loss: 2.1039
Epoch 2/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.3589 - loss: 1.6197
Epoch 2: val_accuracy improved from 0.24882 to 0.38966, saving model to best_mood_model.h5


449/449 ━━━━━━━━━━━━━━━━━━━━ 23s 51ms/step - accuracy: 0.3589 - loss: 1.6196 - val_accuracy: 0.3897 - val_loss: 1.5499
Epoch 3/50
448/449 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.4061 - loss: 1.5264
Epoch 3: val_accuracy improved from 0.38966 to 0.43703, saving model to best_mood_model.h5


449/449 ━━━━━━━━━━━━━━━━━━━━ 24s 54ms/step - accuracy: 0.4061 - loss: 1.5263 - val_accuracy: 0.4370 - val_loss: 1.4793
Epoch 4/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.4329 - loss: 1.4507
Epoch 4: val_accuracy improved from 0.43703 to 0.44121, saving model to best_mood_model.h5


...

449/449 ━━━━━━━━━━━━━━━━━━━━ 24s 53ms/step - accuracy: 0.5631 - loss: 1.1460 - val_accuracy: 0.5699 - val_loss: 1.1384
Epoch 14/50
448/449 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.5787 - loss: 1.1132
Epoch 14: val_accuracy did not improve from 0.56994
449/449 ━━━━━━━━━━━━━━━━━━━━ 24s 53ms/step - accuracy: 0.5787 - loss: 1.1132 - val_accuracy: 0.5582 - val_loss: 1.1791
Epoch 15/50
448/449 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.5905 - loss: 1.0937
Epoch 15: val_accuracy did not improve from 0.56994
449/449 ━━━━━━━━━━━━━━━━━━━━ 24s 53ms/step - accuracy: 0.5905 - loss: 1.0938 - val_accuracy: 0.5255 - val_loss: 1.2239
Epoch 16/50
448/449 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5934 - loss: 1.0682
Epoch 16: val_accuracy did not improve from 0.56994
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 55ms/step - accuracy: 0.5934 - loss: 1.0683 - val_accuracy: 0.5346 - val_loss: 1.2965
Epoch 17/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.6164 - loss: 1.0208
Epoch 17: val_ac

449/449 ━━━━━━━━━━━━━━━━━━━━ 24s 54ms/step - accuracy: 0.6095 - loss: 1.0303 - val_accuracy: 0.5800 - val_loss: 1.1275
Epoch 19/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.6187 - loss: 1.0030
Epoch 19: val_accuracy did not improve from 0.57997
449/449 ━━━━━━━━━━━━━━━━━━━━ 24s 53ms/step - accuracy: 0.6187 - loss: 1.0030 - val_accuracy: 0.5722 - val_loss: 1.1836
Epoch 20/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.6277 - loss: 0.9917
Epoch 20: val_accuracy improved from 0.57997 to 0.59069, saving model to best_mood_model.h5


449/449 ━━━━━━━━━━━━━━━━━━━━ 23s 52ms/step - accuracy: 0.6277 - loss: 0.9917 - val_accuracy: 0.5907 - val_loss: 1.1409
Epoch 21/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.6361 - loss: 0.9593
Epoch 21: val_accuracy improved from 0.59069 to 0.59989, saving model to best_mood_model.h5


449/449 ━━━━━━━━━━━━━━━━━━━━ 24s 54ms/step - accuracy: 0.6361 - loss: 0.9593 - val_accuracy: 0.5999 - val_loss: 1.1058
Epoch 22/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6427 - loss: 0.9412
Epoch 22: val_accuracy did not improve from 0.59989
449/449 ━━━━━━━━━━━━━━━━━━━━ 23s 51ms/step - accuracy: 0.6427 - loss: 0.9412 - val_accuracy: 0.5730 - val_loss: 1.1904
Epoch 23/50
448/449 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.6527 - loss: 0.9209
Epoch 23: val_accuracy did not improve from 0.59989
449/449 ━━━━━━━━━━━━━━━━━━━━ 24s 54ms/step - accuracy: 0.6527 - loss: 0.9209 - val_accuracy: 0.5784 - val_loss: 1.1571
Epoch 24/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.6641 - loss: 0.8878
Epoch 24: val_accuracy did not improve from 0.59989
449/449 ━━━━━━━━━━━━━━━━━━━━ 24s 53ms/step - accuracy: 0.6641 - loss: 0.8878 - val_accuracy: 0.5957 - val_loss: 1.1559
Epoch 25/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.6695 - loss: 0.8863
Epoch 25: val_ac

449/449 ━━━━━━━━━━━━━━━━━━━━ 24s 52ms/step - accuracy: 0.6695 - loss: 0.8863 - val_accuracy: 0.6032 - val_loss: 1.1566
Epoch 26/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.6749 - loss: 0.8583
Epoch 26: val_accuracy did not improve from 0.60323
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 56ms/step - accuracy: 0.6749 - loss: 0.8583 - val_accuracy: 0.6025 - val_loss: 1.1756


In [12]:
# Evaluate model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")


113/113 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.4953 - loss: 1.3171
Test Accuracy: 59.99%
Test Loss: 1.1058


In [18]:
import os

test_dir = os.path.join(path, "test")  # correct path


In [36]:
import os, random
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load model
model = load_model("best_mood_model.h5")
class_labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

# Path to your dataset test folder
test_dir = "/content/IMG_4515.JPG"
# Choose a random emotion folder (e.g. happy, sad...)
#emotion_folder = random.choice(os.listdir(test_dir))

# Choose a random image from that folder
#img_name = random.choice(os.listdir(os.path.join(test_dir, emotion_folder)))
#img_path = os.path.join(test_dir, emotion_folder, img_name)

print("📷 Testing on:", test_dir)

# Load and preprocess the image
img = cv2.imread(test_dir, cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (48,48))
img = img.astype("float32")/255.0
img = np.expand_dims(img, axis=0)
img = np.expand_dims(img, axis=-1)

# Predict
prediction = model.predict(img)
mood = class_labels[np.argmax(prediction)]
print("✅ Predicted Mood:", mood)


📷 Testing on: /content/IMG_4515.JPG
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step
✅ Predicted Mood: Surprise


In [32]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load trained model
model = load_model("best_mood_model.h5")

# Emotion labels
class_labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

# Load face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Extract face ROI
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48,48))
        roi = roi_gray.astype("float32") / 255.0
        roi = np.expand_dims(roi, axis=0)
        roi = np.expand_dims(roi, axis=-1)

        # Predict emotion
        preds = model.predict(roi)
        label = class_labels[np.argmax(preds)]

        # Draw rectangle and label
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(frame, label, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

    cv2.imshow('Live Mood Detector', frame)

    # Press Q to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [33]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL.Image

def js_to_image(js_reply):
    """Convert js object to OpenCV image"""
    image_bytes = b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    return cv2.imdecode(jpg_as_np, flags=1)

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
      async function takePhoto(quality) {
        const div = document.createElement('div');
        const capture = document.createElement('button');
        capture.textContent = 'Capture';
        div.appendChild(capture);

        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // Resize the output to fit the video element.
        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        // Wait for Capture to be clicked.
        await new Promise((resolve) => capture.onclick = resolve);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getTracks().forEach(track => track.stop());
        div.remove();
        return canvas.toDataURL('image/jpeg', quality);
      }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    img = js_to_image(data)
    cv2.imwrite(filename, img)
    return filename, img
